In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


['data.npy', 'labels.npy']


In [2]:

import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K





class alexnet:
    @staticmethod
    def build(width,height,depth,classes):
        model=Sequential()
        inputS=(height,width,depth)
        chanDim=-1
        if(K.image_data_format()=="channels_first"):
            inputS=(depth,height,width)
            chanDim=1

        model.add(Conv2D(filters=128, input_shape=inputS, kernel_size=(4,4), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(Conv2D(filters=256, kernel_size=(4,4), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(Flatten())
        model.add(Dense(4096, input_shape=(128*128*3,)))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.2))
        model.add(Dense(4096))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1000))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        return model




Using TensorFlow backend.


In [3]:
!pip install imutils
import matplotlib
matplotlib.use("Agg")

import tensorflow as tf
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
from keras import backend as K

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

EPOCHS = 32
INIT_LR = 1e-3 #Initial Learning rate
BS = 32 # Bach size to feed

print("Load images' NPY file")
data = []
labels = []

random.seed(42)


tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
data=np.load('../input/data.npy')
labels=np.load('../input/labels.npy')

(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=6)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest") 

print("Compiling model...")
model = alexnet.build(128,128,3,2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) 
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

print("Training network")

H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX)//BS,
	epochs=EPOCHS, verbose=1,callbacks=[tensorboard])

label_name=["benign","malicious"]


print("Save the model for the applied CNN")



print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128) #Check this
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1)))

cm = confusion_matrix(testY.argmax(axis=1), predictions.argmax(axis=1))
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

plt.style.use("ggplot")
plt.figure()
N = EPOCHS

K.clear_session()

  Stored in directory: /tmp/.cache/pip/wheels/b2/40/59/139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built imutils
Load images' NPY file
Compiling model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training network
Instructions for updating:
Use tf.cast instead.
Epoch 1/32
52/52 [==============================] - 12s 222ms/step - loss: 7.8836 - acc: 0.4982 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 2/32
52/52 [==============================] - 8s 150ms/step - loss: 8.0898 - acc: 0.4953 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 3/32
52/52 [==============================] - 8s 149ms/step - loss: 7.9742 - acc: 0.5026 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 4/32
52/52 [==============================] - 8s 146ms/step - loss: 8.1669 - acc: 0.4905 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 5/32
52/52 [===========================

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
